In [1]:
# for webscraping
from googlesearch import search
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# !pip install beautifulsoup4
# !pip install googlesearch-python

In [3]:
bias_rating_url = "https://www.allsides.com/media-bias/media-bias-ratings"

In [4]:
bias_page = requests.get(bias_rating_url)
bias_soup = BeautifulSoup(bias_page.content, 'html.parser')

In [5]:
# webscrape the sources name and bias

content = bias_soup.find_all(id = 'content')[0]
table = content.find_all(class_ = 'views-table cols-4')[0]

source_titles = table.tbody.find_all(class_ = 'views-field views-field-title source-title')
source_biases = table.tbody.find_all(class_ = 'views-field views-field-field-bias-image')

In [6]:
news_sources = []
news_biases = []
news_homepage = []
for i in range(len(source_titles)):
    source_name = source_titles[i].a.text
    # common sources
    if (source_name == 'CBS News (Online)') or (
        source_name == "CNN (Opinion)") or (
        source_name == 'Fox News (Opinion)') or (
        source_name == 'AP Politics & Fact Check') or (
        source_name == 'New York Times (Opinion)') or (
        source_name == 'NPR (Opinion)') or (
        source_name == 'Wall Street Journal (Opinion)'):
        continue
    news_sources.append(source_name)
    news_url = next(search(source_titles[i].a.text))
    news_url =  re.findall(r"^.+\.com|^.+\.org|^.+\.uk", news_url)[0]
    news_homepage.append(news_url)
    news_biases.append(source_biases[i].a.get('href').strip('/media-bias/').split("-")[0])

In [7]:
# put the news sources and biases in a dataframe: data
data = pd.DataFrame(data = {'source':news_sources,
                            'website':news_homepage,
                            'bias': news_biases})

In [8]:
data.head()

,source,website,bias
0,ABC News (Online),https://abcnews.go.com,left
1,AlterNet,https://www.alternet.org,left
2,Associated Press,https://apnews.com,center
3,Axios,https://www.axios.com,center
4,BBC News,https://www.bbc.com,center


In [9]:
data.to_csv('bias_df.csv')